In [1]:
import time

B = BooleanPolynomialRing(64, 'x', order='degrevlex')
B.inject_variables()

with open('V1FF.txt') as file:  
    f_function = file.read().replace('_{', '').replace('} x', '*x').replace('} +', '+')
    f_function = eval(f_function)
    
with open('V1G.txt') as file:  
    gamma_bits = file.read()
    seq_gamma = [int(bit) for bit in gamma_bits.strip()]

print(f_function)
print("_________________________________________________________")
print(seq_gamma[:20])
print("_________________________________________________________")


feedback_poly = B['x']('x^64 + x^63 + x^62 + x^60 + x^59 + x^58 + x^57 + x^56 + x^53 + '
                  'x^50 + x^47 + x^45 + x^44 + x^43 + x^42 + x^41 + x^40 + x^39 + '
                  'x^38 + x^37 + x^36 + x^34 + x^32 + x^30 + x^28 + x^24 + x^18 + '
                  'x^15 + x^14 + x^13 + x^11 + x^9 + x^6 + x^4 + 1')

start_time = time.time()
I = Ideal(f_function)
GB = I.groebner_basis()
print("Groebner Basis for I:", GB)
print("_________________________________________________________")
end_time = time.time()
print("Time: {:.2f} seconds.".format(end_time - start_time))
print("_________________________________________________________")


start_time = time.time()
I_1 = Ideal(f_function + 1)
GB_1 = I_1.groebner_basis()
print("Groebner Basis for I + 1:", GB_1)
print("_________________________________________________________")
end_time = time.time()
print("Time: {:.2f} seconds.".format(end_time - start_time))

Defining x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, x35, x36, x37, x38, x39, x40, x41, x42, x43, x44, x45, x46, x47, x48, x49, x50, x51, x52, x53, x54, x55, x56, x57, x58, x59, x60, x61, x62, x63
x57*x56*x53*x52*x49*x48*x46*x41*x39*x38*x37*x29*x26*x25*x23*x20*x6*x5*x3*x1*x0 + x57*x53*x52*x49*x48*x47*x46*x41*x39*x38*x37*x29*x26*x25*x23*x20*x6*x3*x1*x0 + x57*x53*x52*x49*x48*x47*x46*x41*x39*x37*x29*x26*x25*x23*x20*x6*x3*x1*x0 + x57*x53*x52*x49*x48*x46*x41*x39*x31*x29*x26*x25*x23*x20*x6*x3*x1*x0 + x53*x52*x49*x48*x46*x41*x39*x31*x29*x26*x25*x23*x20*x6*x3*x1*x0 + x53*x52*x49*x48*x46*x41*x39*x29*x25*x23*x20*x14*x6*x3*x1*x0 + x53*x52*x49*x48*x46*x41*x39*x29*x26*x25*x23*x20*x6*x3*x1*x0 + x53*x52*x49*x48*x41*x39*x29*x25*x23*x20*x14*x6*x3*x1*x0 + x53*x52*x49*x48*x41*x39*x34*x29*x25*x23*x20*x6*x3*x1*x0 + x53*x52*x49*x48*x46*x41*x39*x29*x25*x23*x20*x6*x3*x1*x0 + x53*x52*x49*x41

In [2]:
def get_min_degree_polynomials(GB, print_result=True):
    if not GB:
        return []

    min_degree = min(poly.degree() for poly in GB)
    min_polys = [poly for poly in GB if poly.degree() == min_degree]
    
    if print_result:
        print(f"Found {len(min_polys)} polynomial(s) of minimum degree {min_degree}:")
        for i, poly in enumerate(min_polys, 1):
            print(f"{i}: {poly}")
        print("_____________________________________________________")
    
    return min_polys


start_time = time.time()
min_polys_GB = get_min_degree_polynomials(GB)
min_polys_GB_1 = get_min_degree_polynomials(GB_1)
end_time = time.time()

print("Time for finding min-degree polynomials: {:.2f} seconds".format(end_time - start_time))

min_poly_f = min_polys_GB[0]
min_poly_f1 = min_polys_GB_1[0]
print(f"(from GB): {min_poly_f}")
print(f"(from GB_1): {min_poly_f1}")


Found 1 polynomial(s) of minimum degree 2:
1: x53*x0 + x0 + x53 + 1
_____________________________________________________
Found 1 polynomial(s) of minimum degree 2:
1: x53*x0 + x53
_____________________________________________________
Time for finding min-degree polynomials: 0.02 seconds
(from GB): x53*x0 + x0 + x53 + 1
(from GB_1): x53*x0 + x53


In [3]:
def gen_equations(C, init_state, min_poly, min_poly_plus1, gamma_bits):
    equations = []
    state = init_state
    for bit in gamma_bits:
        state = C * state
        poly = min_poly_plus1 if bit == 1 else min_poly
        equations.append(poly(*state))
    return equations

C = companion_matrix(feedback_poly, format='bottom')
init_state = vector(B, [B(f'x{i}') for i in range(64)])

print("_________________________________________________________")
start_time = time.time()
equations = gen_equations(C, init_state, min_poly_f, min_poly_f1, seq_gamma[:1000])
end_time = time.time()
print("Time: {:.2f} seconds.".format(end_time - start_time))
print("_________________________________________________________")

for eq in equations[:10]:
    print(eq)


_________________________________________________________
Time: 2.74 seconds.
_________________________________________________________
x54*x1 + x54
x55*x2 + x2 + x55 + 1
x56*x3 + x56
x57*x4 + x4 + x57 + 1
x58*x5 + x5 + x58 + 1
x59*x6 + x6 + x59 + 1
x60*x7 + x60
x61*x8 + x8 + x61 + 1
x62*x9 + x62
x63*x10 + x63


In [4]:
start_time = time.time()
GB_start = Ideal(equations).groebner_basis()
end_time = time.time()
print("Time: {:.2f} seconds.".format(end_time - start_time))
print("_________________________________________________________")

for i in GB_start:
	print(i)

Time: 54.18 seconds.
_________________________________________________________
x0
x1 + 1
x2
x3
x4 + 1
x5 + 1
x6 + 1
x7
x8
x9 + 1
x10 + 1
x11 + 1
x12 + 1
x13
x14 + 1
x15
x16
x17
x18 + 1
x19 + 1
x20
x21 + 1
x22
x23
x24 + 1
x25 + 1
x26
x27 + 1
x28 + 1
x29
x30 + 1
x31 + 1
x32 + 1
x33 + 1
x34 + 1
x35 + 1
x36 + 1
x37 + 1
x38
x39 + 1
x40
x41
x42 + 1
x43 + 1
x44
x45 + 1
x46 + 1
x47 + 1
x48
x49 + 1
x50 + 1
x51
x52 + 1
x53 + 1
x54
x55 + 1
x56
x57 + 1
x58 + 1
x59 + 1
x60
x61 + 1
x62
x63 + 1


In [6]:
binary_string = ['0' for i in range(64)]

for poly in GB_start:
    poly_str = str(poly).replace(' ', '')  
    if poly_str.startswith('x'):
        if '+1' in poly_str:
            index = int(poly_str[1:].split('+')[0])
            binary_string[index] = '1'
        else:
            index = int(poly_str[1:])
            binary_string[index] = '0'

binary_string = ''.join(binary_string)
print("Recovered binary string:", binary_string)

Recovered binary string: 0100111001111010001101001101101111111101001101110110110101110101


In [7]:
test_state = vector(B, binary_string)
state_is_recovered = True

start_time = time.time()
for i in range(len(seq_gamma)):
    test_state = C * test_state
    expected_bit = int(seq_gamma[i])
    computed_bit = int(f_function(*test_state))
    
    if expected_bit != computed_bit:
        print(f"Mismatch at position {i}: expected {expected_bit}, got {computed_bit}")
        state_is_recovered = False
        break
end_time = time.time()
print("Time: {:.2f} seconds.".format(end_time - start_time))
print("_________________________________________________________")
print(f"State is recovered correctly: {state_is_recovered}")



Time: 106.43 seconds.
_________________________________________________________
State is recovered correctly: True
